In [1]:
#!/usr/bin/env python3
# ipa_analyzer.py

import zipfile
import os
import subprocess
import plistlib
from pathlib import Path

class IPAAnalyzer:
    def __init__(self, ipa_path):
        self.ipa_path = ipa_path
        self.temp_dir = "/tmp/ipa_analysis"
        self.app_bundle = None
        
    def extract_ipa(self):
        """Extract IPA contents"""
        os.makedirs(self.temp_dir, exist_ok=True)
        
        with zipfile.ZipFile(self.ipa_path, 'r') as zip_ref:
            zip_ref.extractall(self.temp_dir)
            
        # Find the .app bundle
        payload_dir = os.path.join(self.temp_dir, "Payload")
        for item in os.listdir(payload_dir):
            if item.endswith('.app'):
                self.app_bundle = os.path.join(payload_dir, item)
                break
                
        return self.app_bundle
    
    def analyze_binary(self):
        """Analyze the main executable"""
        if not self.app_bundle:
            return None
            
        # Find main executable
        info_plist = os.path.join(self.app_bundle, "Info.plist")
        with open(info_plist, 'rb') as f:
            plist_data = plistlib.load(f)
            
        executable_name = plist_data.get('CFBundleExecutable')
        executable_path = os.path.join(self.app_bundle, executable_name)
        
        # Analyze with lipo and otool
        analysis = {
            'executable_path': executable_path,
            'architectures': self._get_architectures(executable_path),
            'dependencies': self._get_dependencies(executable_path),
            'bundle_info': plist_data
        }
        
        return analysis
    
    def _get_architectures(self, binary_path):
        """Get binary architectures"""
        try:
            result = subprocess.run(['lipo', '-info', binary_path], 
                                  capture_output=True, text=True)
            return result.stdout.strip()
        except:
            return "Unknown"
    
    def _get_dependencies(self, binary_path):
        """Get framework dependencies"""
        try:
            result = subprocess.run(['otool', '-L', binary_path], 
                                  capture_output=True, text=True)
            dependencies = []
            for line in result.stdout.split('\n')[1:]:
                if line.strip() and not line.startswith('\t/usr/lib/libobjc'):
                    dep = line.strip().split(' ')[0]
                    dependencies.append(dep)
            return dependencies
        except:
            return []

# Test your IPA
if __name__ == "__main__":
    analyzer = IPAAnalyzer("/Users/himanshukukreja/autoflow/ios-bridge/BitBarSampleApp.ipa")
    app_bundle = analyzer.extract_ipa()
    analysis = analyzer.analyze_binary()
    
    print("=== IPA Analysis ===")
    print(f"App Bundle: {app_bundle}")
    print(f"Architectures: {analysis['architectures']}")
    print(f"Dependencies: {analysis['dependencies'][:5]}...")  # First 5

=== IPA Analysis ===
App Bundle: /tmp/ipa_analysis/Payload/BitBarSampleApp.app
Architectures: Non-fat file: /tmp/ipa_analysis/Payload/BitBarSampleApp.app/BitBarSampleApp is architecture: arm64
Dependencies: ['/usr/lib/libc++.1.dylib', '/usr/lib/libc++abi.dylib', '/usr/lib/libicucore.A.dylib', '/System/Library/Frameworks/AudioToolbox.framework/AudioToolbox', '/System/Library/Frameworks/CFNetwork.framework/CFNetwork']...
